In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/train_spam.csv')
data.head()

,text_type,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...


# Анализ и предобработка данных

In [3]:
# заменим spam на 1 и ham на 0
data.loc[data['text_type'] == 'spam', 'text_type'] = 1
data.loc[data['text_type'] == 'ham', 'text_type'] = 0
data['text_type'] = data['text_type'].astype(int)

In [4]:
# посмотрим на пропуски
data.isna().sum()

text_type    0
text         0
dtype: int64

In [5]:
# посмотрим на сбаллансированность данных
data.groupby(by='text_type').count()

,text
text_type,
0,11469
1,4809


In [6]:
# предобработаем текста

import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')


def clear_text(text: str, method=None):
    """
    Функция для предобработки текста

    Parameters
    ----------
    text
    method:
        - 'lemmatize': использовать WordNetLemmatizer()
        - 'stem': использовать PorterStemmer()
        - None: не использовать обработку слов

    Returns
    -------
    Текст, готовый к векторизации
    """
    stopwords_set = set(stopwords.words('english'))

    text = text.strip().lower()
    text = text.translate(str.maketrans('', '', string.punctuation))  # удаление пунктуации
    text = re.sub(r'\d+', '', text)  # удаление чисел
    text_list = text.split()
    text_list = [word for word in text_list if word not in stopwords_set]
    if method == 'lemmatize':
        lemmatizer = WordNetLemmatizer()
        text_list = [lemmatizer.lemmatize(word) for word in text_list]
    elif method == 'stem':
        stemmer = PorterStemmer()
        text_list = [stemmer.stem(word) for word in text_list]
    text = ' '.join(text_list)
    return text


data['cleared_text'] = data['text'].apply(clear_text)
data['cleared_text_stem'] = data['text'].apply(clear_text, method='stem')
data['cleared_text_lemmatize'] = data['text'].apply(clear_text, method='lemmatize')
data.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kudr.max/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kudr.max/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text_type,text,cleared_text,cleared_text_stem,cleared_text_lemmatize
0,0,make sure alex knows his birthday is over in f...,make sure alex knows birthday fifteen minutes ...,make sure alex know birthday fifteen minut far...,make sure alex know birthday fifteen minute fa...
1,0,a resume for john lavorato thanks vince i will...,resume john lavorato thanks vince get moving r...,resum john lavorato thank vinc get move right ...,resume john lavorato thanks vince get moving r...
2,1,plzz visit my website moviesgodml to get all m...,plzz visit website moviesgodml get movies free...,plzz visit websit moviesgodml get movi free al...,plzz visit website moviesgodml get movie free ...
3,1,urgent your mobile number has been awarded wit...,urgent mobile number awarded £ prize guarantee...,urgent mobil number award £ prize guarante cal...,urgent mobile number awarded £ prize guarantee...
4,0,overview of hr associates analyst project per ...,overview hr associates analyst project per dav...,overview hr associ analyst project per david r...,overview hr associate analyst project per davi...


In [7]:
X = data['cleared_text_stem']
y = data['text_type']

In [8]:
# векторизируем текст

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tfidf_vect = TfidfVectorizer()
# count_vect = CountVectorizer()

tfidf_vect.fit(X_train)
# count_vect.fit(X_train)

X_train_vect = tfidf_vect.transform(X_train)
X_test_vect = tfidf_vect.transform(X_test)
# X_train_vect = count_vect.transform(X_train)
# X_test_vect = count_vect.transform(X_test)

# Обучение модели

Рассмотрим несколько моделей и для каждой из них подберем оптимальные гиперпараметры

In [9]:
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import GridSearchCV
import numpy as np

### Логистическая регрессия

In [19]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

parameters = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

grid_search = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    scoring='roc_auc',
    cv=5)

grid_search.fit(X_train_vect, y_train)

print(f'Лучший roc_auc: {grid_search.best_score_}')
print(f'При параметрах: {grid_search.best_params_}')

Лучший roc_auc: 0.9776756178251041
При параметрах: {'C': 10}


In [20]:
model = grid_search.best_estimator_
model.fit(X_train_vect, y_train)

y_pred = model.predict(X_test_vect)
y_pred_proba = model.predict_proba(X_test_vect)[:, 1]
print(f"ROC-AUC on test: {roc_auc_score(y_test, y_pred_proba)}")
print(f"Classification report:\n{classification_report(y_test, y_pred)}")

ROC-AUC: 0.9819255484105828
Classification report:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      2321
           1       0.91      0.88      0.89       935

    accuracy                           0.94      3256
   macro avg       0.93      0.92      0.93      3256
weighted avg       0.94      0.94      0.94      3256



### SVM

In [ ]:
from sklearn.svm import SVC

model = SVC()

parameters = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['linear', 'rbf', 'poly']
}

grid_search = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    scoring='roc_auc',
    cv=5)

grid_search.fit(X_train_vect, y_train)

print(f'Лучший roc_auc: {grid_search.best_score_}')
print(f'При параметрах: {grid_search.best_params_}')

In [ ]:
model = grid_search.best_estimator_
model.fit(X_train_vect, y_train)

y_pred = model.predict(X_test_vect)
y_pred_proba = model.predict_proba(X_test_vect)[:, 1]
print(f"ROC-AUC on test: {roc_auc_score(y_test, y_pred_proba)}")
print(f"Classification report:\n{classification_report(y_test, y_pred)}")

In [34]:
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, classification_report

model = SVC(kernel='linear', probability=True)
model.fit(X_train_vect, y_train)

SVC(kernel='linear', probability=True)

In [35]:
y_pred = model.predict(X_test_vect)
y_pred_proba = model.predict_proba(X_test_vect)[:, 1]
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba)}")
print(f"Classification report:\n{classification_report(y_test, y_pred)}")

ROC-AUC: 0.9821670080432785
Classification report:
              precision    recall  f1-score   support

           0       0.95      0.96      0.96      2321
           1       0.91      0.89      0.90       935

    accuracy                           0.94      3256
   macro avg       0.93      0.92      0.93      3256
weighted avg       0.94      0.94      0.94      3256



### Градиентный бустинг (CatBoost)

In [58]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score, classification_report

train_pool = Pool(
    data=X_train_vect,
    label=y_train,
)

test_pool = Pool(
    data=X_test_vect,
    label=y_test,
)

catboost_model = CatBoostClassifier(
    metric_period=50,
    use_best_model=True,
    iterations=1100,
    depth=7,
    learning_rate=0.1,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
)

catboost_model.fit(train_pool, eval_set=test_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.6972483	best: 0.6972483 (0)	total: 433ms	remaining: 7m 56s
50:	test: 0.9448228	best: 0.9448228 (50)	total: 9.79s	remaining: 3m 21s
100:	test: 0.9631120	best: 0.9631120 (100)	total: 14.8s	remaining: 2m 26s
150:	test: 0.9714983	best: 0.9714983 (150)	total: 19.9s	remaining: 2m 5s
200:	test: 0.9764681	best: 0.9764681 (200)	total: 24.9s	remaining: 1m 51s
250:	test: 0.9779726	best: 0.9779726 (250)	total: 30s	remaining: 1m 41s
300:	test: 0.9789896	best: 0.9789896 (300)	total: 35.1s	remaining: 1m 33s
350:	test: 0.9797448	best: 0.9797448 (350)	total: 40.1s	remaining: 1m 25s
400:	test: 0.9800856	best: 0.9800856 (400)	total: 45.2s	remaining: 1m 18s
450:	test: 0.9808445	best: 0.9808445 (450)	total: 50.2s	remaining: 1m 12s
500:	test: 0.9808795	best: 0.9808795 (500)	total: 55.3s	remaining: 1m 6s
550:	test: 0.9812297	best: 0.9812297 (550)	total: 1m	remaining: 1m
600:	test: 0.9815228	best: 0.9815228 (600)	total: 1m 5s	remaining: 54.1s
650:	test: 0.9816281	best: 0.9816281 (650)	total: 1m 10s

In [59]:
y_pred = catboost_model.predict(X_test_vect)
y_pred_proba = catboost_model.predict_proba(X_test_vect)[:, 1]
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba)}")
print(f"Classification report:\n{classification_report(y_test, y_pred)}")

ROC-AUC: 0.9828867789331077
Classification report:
              precision    recall  f1-score   support

           0       0.94      0.98      0.96      2321
           1       0.94      0.84      0.89       935

    accuracy                           0.94      3256
   macro avg       0.94      0.91      0.92      3256
weighted avg       0.94      0.94      0.94      3256

